In [1]:
import sys, os
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)
print("Added to sys.path:", repo_root)
from fixedincomelib import *
print("Fixed Income Library is loaded.")

Added to sys.path: /Users/lunli/Documents/FixedIncomeLib
Fixed Income Library is loaded.


### Build a Yield Curve

In [2]:
### de-serialize a pre-built model
path = 'serialized/yc_model.pickle'
yc_model : Model = qfReadModelFromFile(path)
# rebuild model for special dates
value_date = qfDisplayModelValueDate(yc_model)
data_collection = qfGetDataCollectionFromModel(yc_model)
build_method_collection = qfGetBuildMethodCollection(yc_model)
value_date_last_date = '2026-05-26'
yc_model_last_date = qfCreateModel(value_date_last_date, 'YIELD_CURVE', data_collection, build_method_collection)
value_date_expired = '2026-05-27'
yc_model_expired = qfCreateModel(value_date_expired, 'YIELD_CURVE', data_collection, build_method_collection)

### Build Product Bullet Cashflow

In [3]:
### create product bullet cashflow
termination_date = '2026-05-26'
currency = 'USD'
notional = 1e6
long_or_short = 'LONG'
payment_date = qfAddPeriod(termination_date, '2D', 'F', 'USGS')
prod_bullet_cashflow = qfCreateProductBulletCashflow(termination_date, currency, notional, long_or_short)

### Build Valuation Parameter Collection

In [4]:
### create funding index valuation parameters
vp_type = 'FUNDING INDEX PARAMETER'
content = {'Funding Index' : 'SOFR-1B'}
fi_vp = qfCreateValuationParameters(vp_type, content)
vp_collection = qfCreateValuationParametersCollection([fi_vp])

### Test Valuation

In [5]:
### test pv and cash
report = qfCreateValueReport(yc_model, prod_bullet_cashflow, vp_collection, 'pvdetailed')
display(report.display())
report_last_date = qfCreateValueReport(yc_model_last_date, prod_bullet_cashflow, vp_collection, 'pvdetailed')
display(report_last_date.display())
report_expired = qfCreateValueReport(yc_model_expired, prod_bullet_cashflow, vp_collection, 'pvdetailed')
display(report_expired.display())

,Currency,Type,Value
0,USD,PV,987422.235423
1,USD,CASH,0.000000


,Currency,Type,Value
0,USD,PV,0.0
1,USD,CASH,1000000.0


,Currency,Type,Value
0,USD,PV,0.0
1,USD,CASH,0.0


In [6]:
### test risk
rr_report = qfCreateValueReport(yc_model, prod_bullet_cashflow, vp_collection, 'firstorderrisk')
display(rr_report)
rr_report_last_date= qfCreateValueReport(yc_model_last_date, prod_bullet_cashflow, vp_collection, 'firstorderrisk')
display(rr_report_last_date)
rr_report_expired = qfCreateValueReport(yc_model_expired, prod_bullet_cashflow, vp_collection, 'firstorderrisk')
display(rr_report_expired)

[array([-243473.9758577 , -173137.04949881,       0.        ,
              0.        ]),
 array([0., 0., 0.])]

[array([0., 0., 0., 0.]), array([0., 0., 0.])]

[array([0., 0., 0., 0.]), array([0., 0., 0.])]

In [8]:
### test cf report
cf_report = qfCreateValueReport(yc_model, prod_bullet_cashflow, vp_collection, 'cashflowsreport')
display(cf_report.display())
cf_report_last_date= qfCreateValueReport(yc_model_last_date, prod_bullet_cashflow, vp_collection, 'cashflowsreport')
display(cf_report_last_date.display())
cf_report_expired = qfCreateValueReport(yc_model_expired, prod_bullet_cashflow, vp_collection, 'cashflowsreport')
display(cf_report_expired.display())

,PRODUCT_TYPE,VALUATION_ENGINE_TYPE,LEG_ID,CASHFLOW_ID,PAY_OR_RECEIVE,NOTIONAL,PAY_DATE,FORECASTED_AMOUNT,PV,DISCOUNG FACTOR
0,PRODUCT_BULLET_CASHFLOW,ValuationEngineProductBulletCashflow,0,0,1.0,1000000.0,"May 26th, 2026",1000000.0,987422.235423,0.987422


,PRODUCT_TYPE,VALUATION_ENGINE_TYPE,LEG_ID,CASHFLOW_ID,PAY_OR_RECEIVE,NOTIONAL,PAY_DATE,FORECASTED_AMOUNT,PV,DISCOUNG FACTOR
0,PRODUCT_BULLET_CASHFLOW,ValuationEngineProductBulletCashflow,0,0,1.0,1000000.0,"May 26th, 2026",1000000.0,1000000.0,1.0


,PRODUCT_TYPE,VALUATION_ENGINE_TYPE,LEG_ID,CASHFLOW_ID,PAY_OR_RECEIVE,NOTIONAL,PAY_DATE,FORECASTED_AMOUNT,PV,DISCOUNG FACTOR
0,PRODUCT_BULLET_CASHFLOW,ValuationEngineProductBulletCashflow,0,0,1.0,1000000.0,"May 26th, 2026",0.0,0.0,1.0
